In [ ]:
pip freeze | grep gensim
# https://groups.google.com/g/gensim/c/8nobtm9tu-g?pli=1
# Вектора с версии 4.0.0 автоматически подвергаются нормализации

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os

In [ ]:
foo = './drive/MyDrive/rscf/tayga_upos_skipgram_300_2_2019/'
os.listdir(foo)

['freqrnc2011.csv', 'model.bin', 'model.txt', 'README', 'meta.json']

In [ ]:
import gensim
import gensim.models.keyedvectors as word2vec

model=word2vec.KeyedVectors.load_word2vec_format(foo+'model.bin', 
                                                 binary=True, unicode_errors='ignore')
#model=word2vec.KeyedVectors.load_word2vec_format('./drive/MyDrive/model.bin', 
#                                                 binary=True, unicode_errors='ignore')

In [ ]:
i = 0
for w in model.vocab.keys():
  if i == 0:
    print(model[w], w)
  i += 1

In [ ]:
# L2 нормализация
model.init_sims(replace=True)

In [ ]:
high_fr_words = []

f = open(foo+'freqrnc2011.csv', 'r')
for line in f:
  line = line.replace('\n', '')
  check = line.split('\t')
  if check[1] == 's':
     pos = 'NOUN'
     if check[2] != 'Freq(ipm)':
       if float(check[2]) > 7.0:  # если ipm больше 1/3/7/10. Можно и нужно менять.
         high_fr_words.append(check[0]+'_'+pos)
  # if check[1] == 'a':
  #   pos = 'ADJ'
  #   if check[2] != 'Freq(ipm)':
  #     if float(check[2]) > 1.0:  # если ipm больше 1/3/7/10. Можно и нужно менять.
  #       high_fr_words.append(check[0]+'_'+pos)
  # if check[1] == 'adv':
  #   pos = 'ADV'
  #   if check[2] != 'Freq(ipm)':
  #     if float(check[2]) > 1.0:  # если ipm больше 1/3/7/10. Можно и нужно менять.
  #       high_fr_words.append(check[0]+'_'+pos)
  elif check[1] == 'v':
    pos = 'VERB'
    if check[2] != 'Freq(ipm)':
      if float(check[2]) > 7.0:  # если ipm больше 1/3/7/10. Можно и нужно менять.
        high_fr_words.append(check[0]+'_'+pos)

f.close()  # комментарии: ctrl + /

In [ ]:
print(len(high_fr_words))
print(high_fr_words[8])

7435
авиакомпания_NOUN


In [ ]:
print(high_fr_words)

In [ ]:
import numpy as np

In [ ]:
all_vectors = []  # Ваня: добавил в блок повыше фильтр на основе словаря Шарова
all_words = []

for w in model.vocab.keys():
  if w in high_fr_words:
    all_vectors.append(model[w])
    all_words.append(w)

In [ ]:
len(all_vectors)

7418

In [ ]:
arr = np.array(all_vectors)
ws = np.array(all_words)

In [ ]:
arr.shape

(7418, 300)

In [ ]:
frame = []  # Ваня: менять выдачу от 10 до 20
wframe = []

dim = 1
for i in range(0, 300):
  first_row = [dim]
  for x in range(0, 300):
    first_row.append(' ')
  
  frame.append(first_row)

  lst = []

  for max in range(1, 21):  # 11, 21, 31...
    asort = arr[:, i].argsort()
    vect = arr[asort][-max]
    word = ws[asort][-max]
    lst.app
    row = ['Max'+str(max)+': '+word]
    for v in vect:
      row.append(str(v))
    frame.append(row)
    wframe.append(word)

  for min in range(0, 20):  # 10, 20, 30...
    asort = arr[:, i].argsort()
    vect = arr[asort][min]
    word = ws[asort][min]
    row = ['Min'+str(min+1)+': '+word]
    for v in vect:
      row.append(str(v))
    frame.append(row)
    wframe.append(word)
  dim += 1

  print(' '.join(lst))

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
for i in frame:
  print(len(i))

In [ ]:
len(wframe)

In [ ]:
for i in wframe[:20]:
  print(i)

In [ ]:
for i in wframe[20:40]:
  print(i)

In [ ]:
vect = arr[arr[:, 0].argsort()]
print(vect)

In [ ]:
print(frame[11])

In [ ]:
import pandas as pd

df = pd.DataFrame(frame)
df.to_csv('w2v_sorted_dimensions_taiga_ipm7_20_VERB_NOUN_norm.tsv', sep='\t', index=False, header=False)